In [36]:
import json
import glob
import csv
import bs4
import re
import os
import subprocess
from openpyxl import load_workbook
import pandas as pd
import pandas_datareader as pdr
import numpy as np
from selenium import webdriver
import yfinance as yf
import time
import datetime
from IPython.display import display

In [2]:
DFList:list=[pd.DataFrame] * 5;
#perform arithmetic operations on date
#print(datetime.datetime.today().strftime('%d-%m-%Y')-datetime.timedelta(days=1));
print(type(datetime.datetime.today()-datetime.timedelta(days=2)));
DateConvert_Str:datetime.datetime=datetime.datetime.today()-datetime.timedelta(days=1);
DateString:str=datetime.datetime.strftime(DateConvert_Str,'%d-%m-%Y');
ThaiSS_FileStr:str='/Users/cedricyang/Downloads/Short Sales_'+DateString+' to '+DateString+'.xlsx';
#or implement checks to see if the files was downloaded successfully
DirListFile:list=os.listdir('/Users/cedricyang/Downloads/');
List_of_file:list=glob.glob('/Users/cedricyang/Downloads/*.xlsx');
NameString:str='/Users/cedricyang/Downloads/';

<class 'datetime.datetime'>


In [29]:
FileIO=open('IRSG.json','r');
DictObjs:dict=json.load(FileIO);
DFList[0]:pd.DataFrame=pd.json_normalize(DictObjs,record_path=['result','records']);
Sora_Quantile=DFList[0]['comp_sora_3m'].quantile([0.25,0.5,0.75,0.95,1.0]).to_dict();
DFList[0]['Indicator 3M > 1M']=DFList[0]['comp_sora_3m']>DFList[0]['comp_sora_1m'];
DFList[0]['Indicator 3M > 6M']=DFList[0]['comp_sora_3m']>DFList[0]['comp_sora_6m'];
display(DFList[0])

,end_of_day,comp_sora_1m,comp_sora_3m,comp_sora_6m,aggregate_volume,timestamp,Indicator 3M > 1M,Indicator 3M > 6M
0,2023-04-18,NaN,NaN,NaN,NaN,1681833375,False,False
1,2023-04-17,3.5390,3.5898,3.3247,963.0,1681833375,True,True
2,2023-04-14,3.5584,3.5928,3.3233,2705.0,1681833375,True,True
3,2023-04-13,3.5791,3.6047,3.3209,1523.0,1681833375,True,True
4,2023-04-12,3.5939,3.6106,3.3213,1726.0,1681833375,True,True
...,...,...,...,...,...,...,...,...
495,2021-04-28,0.1215,0.1856,0.1776,1125.0,1681833375,True,True
496,2021-04-27,0.1287,0.1867,0.1773,2410.0,1681833375,True,True
497,2021-04-26,0.1355,0.1858,0.1769,2557.0,1681833375,True,True
498,2021-04-23,0.1457,0.1864,0.1771,1680.0,1681833375,True,True


In [51]:
Custom_Options=webdriver.ChromeOptions();
Custom_Options.add_argument("--lang=en");
prefs = {
  "translate_whitelists":{"th":"en"},
  "translate":{"enabled":"true"}
};
print(prefs.keys());
Custom_Options.add_experimental_option("prefs", prefs);
ThaiSetTrade:webdriver.chrome.webdriver.WebDriver=webdriver.Chrome(options=Custom_Options);
ThaiSetTrade.get('https://www.settrade.com/th/home');
#//*[@id="modalFloatingAds"]/div/div[1]/button/svg
ThaiSetTrade.find_element(webdriver.common.by.By.XPATH,'//*[@id="modalFloatingAds"]/div/div[1]/button').click();

dict_keys(['translate_whitelists', 'translate'])


In [31]:
#use firefox to avoid issue while code is running and im surfing the net
DriverObjs:webdriver.firefox.webdriver.WebDriver=webdriver.Firefox();
DriverObjs.get('https://www.set.or.th/en/market/statistics/short-sell');
time.sleep(2);
print(DriverObjs.title);
DriverObjs.find_element(webdriver.common.by.By.XPATH,'//*[@id=\"__layout\"]/div/div[2]/div[3]/div/div/div[2]/div/div[2]/button').click();
pass; #similar to __asm__("nop")
time.sleep(2);
DriverObjs.close();

Short Selling - The Stock Exchange of Thailand


In [56]:
FileDate:datetime.date=datetime.date.today()-datetime.timedelta(1);
DateConcate:str=datetime.datetime.strftime(FileDate,'%d-%m-%Y');
BackupFileStr:str='/Users/cedricyang/Downloads/Short\ Sales_'+DateConcate+'\ to\ '+DateConcate+'.xlsx';
print(BackupFileStr);
TerminalCmd='open -a /Applications/Microsoft\ Excel.app ' + BackupFileStr;
subprocess.call('open -a /Applications/Microsoft\ Excel.app '+BackupFileStr,shell=True);

/Users/cedricyang/Downloads/Short\ Sales_18-04-2023\ to\ 18-04-2023.xlsx


In [33]:
LoopList:list=glob.glob('/Users/cedricyang/Downloads/*.xlsx');
Regex:re.Pattern=re.compile('/Users/cedricyang/Downloads/Short Sales?');
matches:list=[];
matches=[stringval for stringval in LoopList if re.match(Regex,stringval)];
print(matches);
#using Mac Terminal ot open file
#os.system('cd //');
#os.chdir('/Users/cedricyang/Downloads/');
#os.chdir('/Users/cedricyang/C3express/');
print(os.curdir)
print(os.system('ls -lt | head -5'));

['/Users/cedricyang/Downloads/Short Sales_18-04-2023 to 18-04-2023.xlsx', '/Users/cedricyang/Downloads/Short Sales_17-04-2023 to 17-04-2023.xlsx']
.
total 2016
-rw-r--r--  1 cedricyang  staff  154927 Apr 19 09:14 SecuritiesLending.ipynb
-rw-r--r--  1 cedricyang  staff  163524 Apr 19 09:13 geckodriver.log
-rw-r--r--@ 1 cedricyang  staff   11921 Apr 18 22:43 HK_Short_Sell_Snapshot_20230418.csv
-rw-r--r--@ 1 cedricyang  staff   11921 Apr 18 22:21 ListofList.csv
0


In [34]:
ThaiSS_DF:pd.DataFrame=pd.read_excel(io=matches[0],engine='openpyxl',header=5,index_col=0);
ThaiSS_DF=ThaiSS_DF[:200];
ThaiSS_DF=ThaiSS_DF.sort_values(by=['Turnover (Baht)','Symbol'],ascending=False)
ThaiSS_DF=ThaiSS_DF.reset_index();
ThaiSS_DF['USD Eqiv Amt']=(ThaiSS_DF['Turnover (Baht)'] / 34.12);
display(ThaiSS_DF[:30])

,Symbol,Volume (Shares),Turnover (Baht),% Short Sale Volume Comparing with Auto Matching,USD Eqiv Amt
0,BANPU,22201700.0,215947850.0,0.159922,6.329069e+06
1,CPALL,2287700.0,143586075.0,0.140486,4.208267e+06
2,GULF,2478900.0,133761950.0,0.180898,3.920339e+06
3,AOT,1780800.0,129161800.0,0.113773,3.785516e+06
4,BDMS,4283900.0,127975475.0,0.175475,3.750747e+06
5,KBANK,877700.0,116573850.0,0.063161,3.416584e+06
6,KBANK-R,829700.0,109956050.0,0.059707,3.222627e+06
7,BANPU-R,9323500.0,89892485.0,0.067158,2.634598e+06
8,BH,350600.0,84263900.0,0.114388,2.469634e+06
9,KTB,4882400.0,82170850.0,0.153450,2.408290e+06


In [53]:
DriverObjs2:webdriver.firefox.webdriver.WebDriver=webdriver.Firefox();
DriverObjs2.get('https://www.set.or.th/en/home');
time.sleep(2);
DriverObjs2.find_element(webdriver.common.by.By.XPATH,'/html/body/div/div/div/header/div[1]/div[1]/div[1]/div[3]/div/div[1]/input').click();
time.sleep(2);
TrendingQuote=DriverObjs2.find_element(webdriver.common.by.By.XPATH,'//*[@id="SearchDropdownSecurities"]');
print(TrendingQuote.text);
time.sleep(2);
DriverObjs2.close();

Popular Quote
Last Update 19 Apr 2023 09:05:18
TISCO
Stock
-
TISCO FINANCIAL GROUP PUBLIC COMPANY LIMITED
- (-)
BANPU
Stock
-
BANPU PUBLIC COMPANY LIMITED
- (-)
PTT
Stock
-
PTT PUBLIC COMPANY LIMITED
- (-)
OR
Stock
-
PTT OIL AND RETAIL BUSINESS PUBLIC COMPANY LIMITED
- (-)
SCB
Stock
-
SCB X PUBLIC COMPANY LIMITED
- (-)


In [14]:
#creating a list of list
ListLoop:list=[list]*630;
print(ListLoop);

[<class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 

In [15]:
HtmlIO_File=open('HK_X_SS.html');
soupObj=bs4.BeautifulSoup(HtmlIO_File,'html.parser');
preTags:str=soupObj.select_one('pre').text;
SplitList:list=preTags.split('\n')[5:];
print(len(SplitList));

624


In [16]:
jjj:int=0;
List_indexer:int=0
for i in range(4,611):
    ListLoop[List_indexer]=SplitList[i].split(" ");
    ListLoop[List_indexer]=[x for x in ListLoop[List_indexer] if x != '']
    List_indexer=List_indexer+1;
    #x for x in list_with_empty_strings if x != ''

LoopCounters:int=0;
OuterList_len:int=len(ListLoop);
#Testing Relational Operator
print(ListLoop[OuterList_len-29][0]);
print(len(ListLoop[OuterList_len-29][0]));
print(ListLoop[OuterList_len-29][0]!='%82800');
print(ListLoop[OuterList_len-29][1]!='TRACKER');
    
while (ListLoop[LoopCounters][0]!='%82800'):
    Each_List_len=len(ListLoop[LoopCounters]);
    if (ListLoop[LoopCounters][0]=='%'): #indicates it's a STOCK traded in NON-HKD
        ListLoop[LoopCounters][Each_List_len-Each_List_len]='NON-HKD Stock/ETF';
        ListLoop[LoopCounters][Each_List_len-1]=int(ListLoop[LoopCounters][Each_List_len-1].replace(',',''));
        ListLoop[LoopCounters][Each_List_len-2]=int(ListLoop[LoopCounters][Each_List_len-2].replace(',',''))
        LoopCounters=LoopCounters+1;
        continue;
    elif (ListLoop[LoopCounters][0]=='*'):
        LoopCounters=LoopCounters+1;
        continue;
    #Stored the converted value back to memory.. slow CPU operations
    ListLoop[LoopCounters][Each_List_len-Each_List_len]=int(ListLoop[LoopCounters][Each_List_len-Each_List_len]);
    ListLoop[LoopCounters][Each_List_len-1]=int(ListLoop[LoopCounters][Each_List_len-1].replace(',',''));
    ListLoop[LoopCounters][Each_List_len-2]=int(ListLoop[LoopCounters][Each_List_len-2].replace(',',''));
    LoopCounters=LoopCounters+1;
    continue;
print(ListLoop);

%83188
6
True
True
[[3, 'HK', '&', 'CHINA', 'GAS', 107000, 737460], [4, 'WHARF', 'HOLDINGS', 21000, 381200], [5, 'HSBC', 'HOLDINGS', 1216800, 68140440], [6, 'POWER', 'ASSETS', 168500, 7339125], [8, 'PCCW', 111000, 455100], [11, 'HANG', 'SENG', 'BANK', 18700, 2144480], [12, 'HENDERSON', 'LAND', 203000, 5517850], [13, 'HUTCHMED', 153000, 4151175], [14, 'HYSAN', 'DEV', 4000, 93000], [16, 'SHK', 'PPT', 209500, 22783400], [17, 'NEW', 'WORLD', 'DEV', 458000, 9673950], [19, 'SWIRE', 'PACIFIC', 'A', 82000, 4809500], [20, 'SENSETIME-W', 7255000, 19945650], [23, 'BANK', 'OF', 'E', 'ASIA', 29600, 296932], [27, 'GALAXY', 'ENT', 383000, 21441000], [35, 'FE', 'CONSORT', 'INTL', 1000, 1980], [38, 'FIRST', 'TRACTOR', 2000, 8040], [45, 'HK&S', 'HOTELS', 500, 3945], [66, 'MTR', 'CORPORATION', 89000, 3458525], [69, 'SHANGRI-LA', 'ASIA', 10000, 74520], [81, 'CH', 'OVS', 'G', 'OCEANS', 247000, 1106380], [83, 'SINO', 'LAND', 52000, 560040], [86, 'SUN', 'HUNG', 'KAI', 'CO', 1000, 3110], [87, 'SWIRE', 'PACIFI

In [17]:
#using "pop" function to retain useful data
def ColLength4(ArgList:list):
    print("Calling 4");
    ArgList.pop(1);
    print(ArgList)
    return;

def ColLength5(ArgList:list):
    print("Calling 5");
    ArgList.pop(1);
    ArgList.pop(1);
    return;

def ColLength6(ArgList:list):
    print("Calling 6");
    ArgList.pop(1);
    ArgList.pop(1);
    ArgList.pop(1);
    return;

def ColLength7(ArgList:list):
    print("Calling 7");
    ArgList.pop(1);
    ArgList.pop(1);
    ArgList.pop(1);
    ArgList.pop(1);
    return;

#creating a dictionary of Function Pointers
CreateFuncPtr:dict=dict.fromkeys([4,5,6,7]);
print(CreateFuncPtr);
CreateFuncPtr[4]=ColLength4;
CreateFuncPtr[5]=ColLength5;
CreateFuncPtr[6]=ColLength6;
CreateFuncPtr[7]=ColLength7;

for LengthIteration in ListLoop:
    print(LengthIteration)
    if (LengthIteration[0]=='NON-HKD Stock/ETF'):
        continue; #for simpliciy skip NON-HKD stocks
    elif (LengthIteration[0]=='%82800'):
        break;
    Find_Len:int=len(LengthIteration);
    CreateFuncPtr[Find_Len](LengthIteration);

print(ListLoop);

{4: None, 5: None, 6: None, 7: None}
[3, 'HK', '&', 'CHINA', 'GAS', 107000, 737460]
Calling 7
[4, 'WHARF', 'HOLDINGS', 21000, 381200]
Calling 5
[5, 'HSBC', 'HOLDINGS', 1216800, 68140440]
Calling 5
[6, 'POWER', 'ASSETS', 168500, 7339125]
Calling 5
[8, 'PCCW', 111000, 455100]
Calling 4
[8, 111000, 455100]
[11, 'HANG', 'SENG', 'BANK', 18700, 2144480]
Calling 6
[12, 'HENDERSON', 'LAND', 203000, 5517850]
Calling 5
[13, 'HUTCHMED', 153000, 4151175]
Calling 4
[13, 153000, 4151175]
[14, 'HYSAN', 'DEV', 4000, 93000]
Calling 5
[16, 'SHK', 'PPT', 209500, 22783400]
Calling 5
[17, 'NEW', 'WORLD', 'DEV', 458000, 9673950]
Calling 6
[19, 'SWIRE', 'PACIFIC', 'A', 82000, 4809500]
Calling 6
[20, 'SENSETIME-W', 7255000, 19945650]
Calling 4
[20, 7255000, 19945650]
[23, 'BANK', 'OF', 'E', 'ASIA', 29600, 296932]
Calling 7
[27, 'GALAXY', 'ENT', 383000, 21441000]
Calling 5
[35, 'FE', 'CONSORT', 'INTL', 1000, 1980]
Calling 6
[38, 'FIRST', 'TRACTOR', 2000, 8040]
Calling 5
[45, 'HK&S', 'HOTELS', 500, 3945]
Callin

In [26]:
#create a daily.csv file
CSV_DateStr:datetime.date=datetime.date.today();
Date_Str:str=datetime.datetime.strftime(CSV_DateStr,'%Y%m%d');
DailyFileName:str='HK_Short_Sell_Snapshot_'+Date_Str+'.csv';
#print(DailyFileName);
os.system('touch '+DailyFileName);
print(os.listdir());

['HK_Short_Sell_Snapshot_20230418.csv', 'HKss.json', 'TestFile.csv', 'geckodriver.log', 'IRSG.json', '.ipynb_checkpoints', 'HK_X_SS.html', 'KoreaX_analytics.ipynb', '.git', 'QueryCmd.sh', 'FedNY.json', 'ListofList.csv', 'ReferenceRate.json', 'SecuritiesLending.ipynb']


In [27]:
with open(DailyFileName, "w", newline="\n") as FileIOs:
    writer = csv.writer(FileIOs,delimiter=',',quoting=csv.QUOTE_MINIMAL);
    writer.writerow(['Tickers','#Number of shs','HKD Notional']);
    #append a header to the CSV file
    for rows in ListLoop:
        if (rows[0]=='%82800'):
            break;
        print(rows); #for easy debugging.. remove print for to speed up
        writer.writerow(rows);

[3, 107000, 737460]
[4, 21000, 381200]
[5, 1216800, 68140440]
[6, 168500, 7339125]
[8, 111000, 455100]
[11, 18700, 2144480]
[12, 203000, 5517850]
[13, 153000, 4151175]
[14, 4000, 93000]
[16, 209500, 22783400]
[17, 458000, 9673950]
[19, 82000, 4809500]
[20, 7255000, 19945650]
[23, 29600, 296932]
[27, 383000, 21441000]
[35, 1000, 1980]
[38, 2000, 8040]
[45, 500, 3945]
[66, 89000, 3458525]
[69, 10000, 74520]
[81, 247000, 1106380]
[83, 52000, 560040]
[86, 1000, 3110]
[87, 2500, 23900]
[101, 502000, 7344460]
[107, 2000, 3820]
[116, 7000, 75100]
[119, 1029000, 2404260]
[123, 535000, 7043540]
[135, 602000, 3844820]
[136, 1252000, 2466360]
[142, 8000, 20160]
[144, 212000, 2501880]
[148, 500, 12100]
[151, 444000, 2304650]
[152, 66500, 467270]
[165, 66000, 381960]
[168, 44000, 3729600]
[175, 1970000, 20683180]
[177, 12000, 93920]
[179, 3000, 27150]
[189, 339000, 2842790]
[200, 311000, 2901540]
[215, 8000, 10840]
[220, 229000, 1770320]
[241, 2218000, 12912540]
[257, 355000, 1183550]
[267, 318000,

In [28]:
os.system('open -a /Applications/Microsoft\ Excel.app/ '+DailyFileName);